# CLUE tutorial

## What does CLUE do?

CLUE (Constrained LUmping for differential Eqiations) implements in Python an algorithm that takes as **input**
* a system of ODEs with polynomial right-hand side
* a list of linear combinations of the unknown functions to be preserved (*observables*)

and **returns** the maximal exact reduction of the system by a linear transformation that preserves the given combinations.

## Basic usage

We will demonstrate CLUE on the following system

$\begin{cases} \dot{x}_1  = x_2^2 + 4x_2x_3 + 4x_3^2,\\ \dot{x}_2  =  4x_3 - 2x_1,\\ \dot{x}_3  = x_1 + x_2 \end{cases}$

with the combination to preserve being just $x_1$.

1. import relevant functions from sympy and the function that does lumping:

In [1]:
from sympy import vring, QQ
from clue import do_lumping

2. Introduce the variables $x_1, x_2, x_3$ by defining the ring of polynomials in these variables (QQ refers to the coefficients being rational numbers; for other options, see below)

In [2]:
R = vring(["x1", "x2", "x3"], QQ)

3. Construct a list of right-hand sides of the ODE. The right-hand sides must be in the same order as the variables in the definition of the ring

In [3]:
ode = [
    x2**2 + 4 * x2 * x3 + 4 * x3**2, # derivative of x1
    4 * x3 - 2 * x1,                 # derivative of x2
    x1 + x2                          # derivative of x3
]

4. Call `do_lumping` providing the system and the combinations to preserve, that is, `[x1]`

In [4]:
do_lumping(ode, [x1])

Original system:
x1 = x2**2 + 4*x2*x3 + 4*x3**2
x2 = -2*x1 + 4*x3
x3 = x1 + x2
Outputs to fix:
[x1]
New variables:
y0 = x1
y1 = x2 + 2*x3
Lumped system:
y0 = y1**2
y1 = 2*y1


{'polynomials': [y1**2, 2*y1], 'subspace': [[1, 0, 0], [0, 1, 2]]}

The computation shows that, in the new variables,

$y_1 = x_1 \quad \text{ and }\quad y_2 = x_2 + 2x_3,$

the system will be reduced to 

$\begin{cases} \dot{y}_1 = y_2^2, \\ \dot{y_2} = 2y_2. \end{cases}$

The `polynomials` part of the output dictionary refers to the right-hand side of the reduced model, and the `subspace` part gives the coefficients for expressing $y_1$ and $y_2$ in terms of $x_1, x_2, x_3$.

*Remark:* to supress the output, use `verbose=False` keyword argument.

## Subtleties and extra options

### Rational numbers

If the model involves rational numbers, e.g. $\frac{2}{3}$, writing `2/3` will yield a floating point number and make the subsequent computation inexact.

We recommend to define a rational number $\frac{p}{q}$ as `QQ(p, q)`. For example, consider the system

$\begin{cases}\dot{x}_1 = x_1 + \frac{x_2}{3},\\ \dot{x}_2 = -\frac{2x_1}{3}, \end{cases}$

where the observable to preserve is $x_1 + x_2$.
The maximal lumping can be found as follows:

In [5]:
R = vring(["x1", "x2"], QQ)
ode = [
    x1 + QQ(1, 3) * x2,
    QQ(-2, 3) * x1
]
do_lumping(ode, [x1 + x2])

Original system:
x1 = x1 + x2/3
x2 = -2*x1/3
Outputs to fix:
[x1 + x2]
New variables:
y0 = x1 + x2
Lumped system:
y0 = y0/3


{'polynomials': [1/3*y0], 'subspace': [[1, 1]]}

### Unknown scalar parameters

It often happens that the system of interest involves unknown scalar parameters, for example:

$\begin{cases}
  \dot{x}_1 = ax_1 + bx_2,\\
  \dot{x}_2 = bx_1 + ax_2
\end{cases}$

with observable $x_1 + x_2$.

We will describe two different ways of applying CLUE to such models.

**Convert the parameters into states with zero derivative**

One can rewrite the system above as

$\begin{cases}
  \dot{x}_1 = a x_1 + b x_2,\\
  \dot{x}_2 = b x_1 + a x_2,\\
  \dot{a} = 0,\\
  \dot{b} = 0.
\end{cases}$

And then apply CLUE:

In [6]:
R = vring(["x1", "x2", "a", "b"], QQ)
ode = [
    a * x1 + b * x2,
    b * x1 + a * x2,
    R(0), # this means that zero is interpreted as zero polynomial, not the zero number
    R(0)
]
do_lumping(ode, [x1 + x2])

Original system:
x1 = a*x1 + b*x2
x2 = a*x2 + b*x1
a = 0
b = 0
Outputs to fix:
[x1 + x2]
New variables:
y0 = x1 + x2
y1 = a + b
Lumped system:
y0 = y0*y1
y1 = 0


{'polynomials': [y0*y1, 0], 'subspace': [[1, 1, 0, 0], [0, 0, 1, 1]]}

Observe that:
* CLUE has also discovered a *reduction for the parameters* suggesting a single macroparameter $y_1 = a + b$
* the new variables will be always linear combinations of the original variables and parameters. If you would like to search for a combination with coefficients involving parameters, use the next method.

**Include symbols $a$ and $b$ into the coefficient field**.

Another option would be to set the coefficient field to be rational functions in the parameters. This method will generally be slower than the previous one because we have an improved version of algorithm for the rational coefficients.

In [7]:
# defining the coefficient field
from sympy import FractionField
coef_field = FractionField(QQ, ["a", "b"])
a, b = coef_field.gens

# defining a system over this field
R = vring(["x1", "x2"], coef_field)
ode = [
    a * x1 + b * x2,
    b * x1 + a * x2
]
do_lumping(ode, [x1 + x2])

Original system:
x1 = a*x1 + b*x2
x2 = a*x2 + b*x1
Outputs to fix:
[x1 + x2]
New variables:
y0 = x1 + x2
Lumped system:
y0 = y0*(a + b)


{'polynomials': [(a + b)*y0], 'subspace': [[1, 1]]}

As has been mentioned above, this method allows to find lumpings in which the new variables are expressed as combinations of the original ones with the *coefficients involving parameters*. For example:

In [8]:
# defining the coefficient field
from sympy import FractionField
coef_field = FractionField(QQ, ["a", "b"])
a, b = coef_field.gens

# defining a system over this field
R = vring(["x1", "x2", "x3"], coef_field)
ode = [
    a * x2 + b * x3,
    x2, 
    x3
]
do_lumping(ode, [x1])

Original system:
x1 = a*x2 + b*x3
x2 = x2
x3 = x3
Outputs to fix:
[x1]
New variables:
y0 = x1
y1 = x2 + b*x3/a
Lumped system:
y0 = a*y1
y1 = y1


{'polynomials': [a*y1, y1], 'subspace': [[1, 0, 0], [0, 1, b/a]]}

### Irrational coefficients

A system may involve irrational coefficients (e.g., $\sqrt{2}$). For example, consider a reaction system from the paper ["A general analysis of exact nonlinear lumping in chemical kinetics" (Example 2)](https://doi.org/10.1016/0009-2509(94)87006-3) by Li and Rabitz:

$\begin{cases}
  \dot{x}_1 = -2 x_1 - 2 x_1 x_2 + 4 x_3 x_4,\\
  \dot{x}_2 = -2 x_2 - 2 x_1 x_2 + 4 x_3 x_4,\\
  \dot{x}_3 = -2 x_3 - 4 x_3 x_4 + 2 x_1 x_2,\\
  \dot{x}_4 = -2 x_4 - 4 x_3 x_4 + 2 x_1 x_2,\\
  \dot{x}_5 = -x_5 + x_1 + 2 x_2 + \sqrt{2} x_6,\\
  \dot{x}_6 = -\sqrt{2} x_6 + 2 x_3 + x_5,\\
  \dot{x}_7 = -\sqrt{2} x_7 + x_1 + x_8,\\
  \dot{x}_8 = -x_8 + 2 x_4 + \sqrt{2} x_7
\end{cases}$

with observable $x_5 - \sqrt{2} x_6$.
In order to apply CLUE to the system, one can include $\sqrt{2}$ into the field of coefficients as follows:

In [9]:
from sympy import sqrt
R = vring([f"x{i + 1}" for i in range(8)], QQ.algebraic_field(sqrt(2)))

ode = [
    -2 * x1 - 2 * x1 * x2 + 4 * x3 * x4,
    -2 * x2 - 2 * x1 * x2 + 4 * x3 * x4,
    -2 * x3 - 4 * x3 * x4 + 2 * x1 * x2,
    -2 * x4 - 4 * x3 * x4 + 2 * x1 * x2,
    -x5 + x1 + 2 * x2 + sqrt(2) * x6,
    -sqrt(2) * x6 + 2 * x3 + x5,
    -sqrt(2) * x7 + x1 + x8,
    -x8 + 2 * x4 + sqrt(2) * x7
]
res = do_lumping(ode, [x8 - sqrt(2) * x7])

Original system:
x1 = -2*x1*x2 - 2*x1 + 4*x3*x4
x2 = -2*x1*x2 - 2*x2 + 4*x3*x4
x3 = 2*x1*x2 - 4*x3*x4 - 2*x3
x4 = 2*x1*x2 - 4*x3*x4 - 2*x4
x5 = x1 + 2*x2 - x5 + sqrt(2)*x6
x6 = 2*x3 + x5 - sqrt(2)*x6
x7 = x1 - sqrt(2)*x7 + x8
x8 = 2*x4 + sqrt(2)*x7 - x8
Outputs to fix:
[-sqrt(2)*x7 + x8]
New variables:
y0 = x1
y1 = x2
y2 = x3
y3 = x4
y4 = x7 - sqrt(2)*x8/2
Lumped system:
y0 = -2*y0*y1 - 2*y0 + 4*y2*y3
y1 = -2*y0*y1 - 2*y1 + 4*y2*y3
y2 = 2*y0*y1 - 4*y2*y3 - 2*y2
y3 = 2*y0*y1 - 4*y2*y3 - 2*y3
y4 = y0 - sqrt(2)*y3 + y4*(-sqrt(2) - 1)
